## Machine Learning for Public Policy HW1 - Data Collection and ETL
Name: **Cathy Chen**

### Part 1: Get Data
**Download the most recent ACS data for every block group in the state**

In [430]:
import pandas as pd
import censusdata
import psycopg2

- Explorate the available ACS APIs from the [website](https://www.census.gov/data/developers/data-sets.html)

In [358]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2019) # Iowa

{'Alabama': censusgeo((('state', '01'),)),
 'Alaska': censusgeo((('state', '02'),)),
 'Arizona': censusgeo((('state', '04'),)),
 'Arkansas': censusgeo((('state', '05'),)),
 'California': censusgeo((('state', '06'),)),
 'Colorado': censusgeo((('state', '08'),)),
 'Delaware': censusgeo((('state', '10'),)),
 'District of Columbia': censusgeo((('state', '11'),)),
 'Connecticut': censusgeo((('state', '09'),)),
 'Florida': censusgeo((('state', '12'),)),
 'Georgia': censusgeo((('state', '13'),)),
 'Idaho': censusgeo((('state', '16'),)),
 'Hawaii': censusgeo((('state', '15'),)),
 'Illinois': censusgeo((('state', '17'),)),
 'Indiana': censusgeo((('state', '18'),)),
 'Iowa': censusgeo((('state', '19'),)),
 'Kansas': censusgeo((('state', '20'),)),
 'Kentucky': censusgeo((('state', '21'),)),
 'Louisiana': censusgeo((('state', '22'),)),
 'Maine': censusgeo((('state', '23'),)),
 'Maryland': censusgeo((('state', '24'),)),
 'Massachusetts': censusgeo((('state', '25'),)),
 'Michigan': censusgeo((('stat

- Extract Iowa data with five variables 

In [403]:
data = censusdata.download('acs5', 2019,
                           censusdata.censusgeo([('state', '19'), ('county', '*'), ('block group', '*')]),
                           ['B23025_002E', 'B15002_011E', 'B15002_016E', 'B25075_002E', 'B25075_010E'])

### Part 2. Transform Data
**Clean and get the data ready to be loaded to the postgres database table**

- Do basic data cleaning for the geographical information. 

In [404]:
data.reset_index(inplace=True)
data['index'] = data['index'].astype('str')
data['index'] = data['index'].str.split(',').tolist()

In [405]:
data['Block_Group'] = data['index'].apply(lambda x: x[0])
data['Block_Group'] = data['Block_Group'].apply(lambda x: x.replace('Block Group ', ''))

In [406]:
data['Census_Tract'] = data['index'].apply(lambda x: x[1])
data['Census_Tract'] = data['Census_Tract'].apply(lambda x: x.replace('Census Tract ', ''))
data['Census_Tract'] = data['Census_Tract'].apply(lambda x: x.lstrip(' '))
data['Census_Tract'] = data['Census_Tract'].apply(lambda x: str(x))

In [408]:
data['County'] = data['index'].apply(lambda x: x[2])
data['County'] = data['County'].apply(lambda x: x.lstrip(' '))
data['County'] = data['County'].apply(lambda x: x.replace(' ', '_'))
data['County'] = data['County'].apply(lambda x: x.replace("'", '_'))

In [409]:
data['State'] =  data['index'].apply(lambda x: x[3])
data['State'] =  data['State'].str.split(':').apply(lambda x: x[0])
data['State'] = data['State'].apply(lambda x: x.lstrip(' '))

- Rename the variables to be understandable.

In [410]:
data = data.rename(columns={"B23025_002E": "Labor_Force", "B15002_011E": "High_School_Graduates_over25", 
                     "B15002_016E": "Master_Graduates_over25", "B25075_002E": "Value_less_than_$10,000", 
                     "B25075_010E": "Value_$50,000_to_59,999"})

In [411]:
data = data.drop(['index'], axis =1)

In [412]:
column_names = ['State', 'County', 'Block_Group', 'Census_Tract', 'Labor_Force', 'High_School_Graduates_over25',
                'Master_Graduates_over25', 'Value_less_than_$10,000', 'Value_$50,000_to_59,999']
data = data.reindex(columns=column_names)

### Part 3. Load Data
**Connect to the postgres and load the data into postgres database table**

- Create table

In [413]:
create_table = 'CREATE TABLE acs.qiyangch_acs_data(\
State varchar(5),\
County varchar(20),\
Block_Group integer,\
Census_Tract varchar(10),\
Labor_Force integer,\
High_School_Graduates_over25 integer,\
Master_Graduates_over25 integer,\
Value_less_than_10000 integer,\
Value_50000_to_59999 integer)'

- Insert information

In [419]:
insert_data = data.apply(lambda x: "INSERT INTO acs.qiyangch_acs_data VALUES ('{}', '{}', {}, '{}', {}, {}, {}, {}, {})".format
                         (x['State'], x['County'], x['Block_Group'], x['Census_Tract'], x['Labor_Force'], x['High_School_Graduates_over25'],
                          x['Master_Graduates_over25'], x['Value_less_than_$10,000'], x['Value_$50,000_to_59,999']), axis = 1)
#for i in insert_data:
#    print(i)

- Connect the database and load data into it

In [421]:
conn = psycopg2.connect(host = 'acs-db.mlpolicylab.dssg.io',
                        port = '5432',
                        database = 'acs_data_loading',
                        user = 'mlpp_student',
                        password = 'CARE-horse-most')

cur = conn.cursor()
cur.execute(create_table)
conn.commit()
cur.close()
conn.close()


In [422]:
conn = psycopg2.connect(host = 'acs-db.mlpolicylab.dssg.io',
                        port = '5432',
                        database = 'acs_data_loading',
                        user = 'mlpp_student',
                        password = 'CARE-horse-most')
cur = conn.cursor()

try:
    for i in insert_data:
        cur.execute(i)
except Exception as e:
    print(e)
    
finally:
    conn.commit()
    cur.close()
    conn.close()
